事前準備:

python 3.12.1

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:
```
pip install openai
```

沒裝tabulate使用:
```
pip install tabulate
```

沒設定key使用:
```
setx OPENAI_API_KEY “<yourkey>”
```
```
echo %OPENAI_API_KEY%
```
可以看你的KEY

In [1]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

創assistant,thread,message!

In [4]:
from tabulate import tabulate

# Upload a file with an "assistants" purpose
file = client.files.create(
    file=open("data\data-aggregated.xlsx", "rb"),
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    name="Flora Customer Service",
    instructions="You are customer service from Flora. when answering any question, you should always check the manual. If question are not in manual, you shouldn't answer and tell them to contact human customer service.",
    tools=[{"type": "retrieval"}],
    model="gpt-3.5-turbo-1106",
    file_ids=[file.id]
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="hey flora! Im having some issue with password! can you reset it for me?"
)


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\super\AppData\Local\Temp\ipykernel_14472\1228887928.py:5: SyntaxWarning: invalid escape sequence '\d'
  file=open("data\data-aggregated.xlsx", "rb"),


APIConnectionError: Connection error.

沒裝tabulate很醜使用

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [22]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                     │
├──────────────┼─────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_edJcJI8UOOziomdBZqJRbK3M                                               │
│ created_at   │ 1704278596                                                                  │
│ description  │                                                                             │
│ file_ids     │ []                                                                          │
│ instructions │ You are a personal math tutor. Write and run code to answer math questions. │
│ metadata     │ {}                                                                          │
│ model        │ gpt-3.5-turbo-1106                                                          │
│ name         │ Math Tutor                       

In [23]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_8coNFccDWTknqX3q6PisiJWX │
│ created_at │ 1704278596                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [24]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ message      │ content                                                                                                                             │
├──────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ msg_o0iTtd7pymlOIwvLW59tUApp                                                                                                        │
│ assistant_id │                                                                                                                                     │
│ content      │ [MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')] │
│ created_at   │ 1704278596                                                                   

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [25]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [30]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

┌─────────────────┬──────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                              │
├─────────────────┼──────────────────────────────────────────────────────────────────────┤
│ id              │ run_QwhSngqYjIywqV4XIblhrugL                                         │
│ assistant_id    │ asst_edJcJI8UOOziomdBZqJRbK3M                                        │
│ cancelled_at    │                                                                      │
│ completed_at    │ 1704353924                                                           │
│ created_at      │ 1704353915                                                           │
│ expires_at      │                                                                      │
│ failed_at       │                                                                      │
│ file_ids        │ []                                                                   │

跑完後打開查看history messages!

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ id           │ msg_o0iTtd7pymlOIwvLW59tUApp                                                                                                        │
│ assistant_id │                                                                                                                                     │
│ content      │ [MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')] │
│ created_at   │ 1704278596                                                                                                                          │
│ file_ids     │ []                                                                                                                                  │
│ metadata     │ {}                                                                           